In [76]:
logaccuracyspread = [.78,.80,.87,.90,.95,.96,.97,.97,.97,.98,.97,.97,.95,.95,.95,.94,.93]
svmaccuracyspread = [.74,.80,.83,.88,.92,.93,.94,.95,.96,.96,.96,.95,.95,.94,.95,.93,.91]
knnaccuracyspread = [.75,.80,.85,.88,.94,.96,.97,.97,.96,.96,.96,.96,.95,.95,.95,.95,.95]
rfaccuracyspread = [.69,.79,.90,.91,.94,.96,.97,.98,.97,.97,.97,.97,.97,.97,.96,.96,.96]

sfsInds = sfs.indices_

feature_list = ['SpectralCentroid', 'SpectralCrestFactor', 'SpectralDecrease', 'SpectralFlatness', 'SpectralFlux',
                    'SpectralRolloff', 'SpectralSkewness', 'SpectralSpread', 'SpectralTonalPowerRatio']

performancePreference = "tradeoff" #highest, fewest, tradeoff

if performancePreference == "highest":
    maxVals = np.array([logaccuracyspread[np.argmax(logaccuracyspread)],
    svmaccuracyspread[np.argmax(svmaccuracyspread)],
    knnaccuracyspread[np.argmax(knnaccuracyspread)],
    rfaccuracyspread[np.argmax(rfaccuracyspread)]])

    numFeats = np.array([np.argmax(logaccuracyspread),
    np.argmax(svmaccuracyspread),
    np.argmax(knnaccuracyspread),
    np.argmax(rfaccuracyspread)])

    maxInds = np.where(maxVals == max(maxVals))
    featInd = [] 
    for i in maxInds[0]:
        featInd.append(numFeats[i])

    leastFeats = np.where(featInd == min(featInd))

    optimalFeatNum = featInd[leastFeats[0][0]] + 1

    if maxInds[0][leastFeats[0][0]] == 0:
        optimalModel = "log"
    elif maxInds[0][leastFeats[0][0]] == 1:
        optimalModel = "svm"
    elif maxInds[0][leastFeats[0][0]] == 2:
        optimalModel = "knn"
    elif maxInds[0][leastFeats[0][0]] == 3:
        optimalModel = "rfa"
elif performancePreference == "fewest":
    lowestVals = [np.argmax(np.array(logaccuracyspread) > .8),
    np.argmax(np.array(svmaccuracyspread) > .8),
    np.argmax(np.array(knnaccuracyspread) > .8),
    np.argmax(np.array(rfaccuracyspread) > .8)]

    minInd = np.where(lowestVals == min(lowestVals))

    optimalFeatNum = lowestVals[minInd[0][0]] + 1

    if minInd[0][0] == 0:
        optimalModel = "log"
    elif minInd[0][0] == 1:
        optimalModel = "svm"
    elif minInd[0][0] == 2:
        optimalModel = "knn"
    elif minInd[0][0] == 3:
        optimalModel = "rfa"
elif performancePreference == "tradeoff":
    for i in range(0,len(logaccuracyspread)-3):
        if logaccuracyspread[i+1] - logaccuracyspread[i] < 0.05 and logaccuracyspread[i+2] - logaccuracyspread[i] < 0.05 and logaccuracyspread[i+3] - logaccuracyspread[i] < 0.05:
            logInd = i
            break
    for i in range(0,len(svmaccuracyspread)-3):
        if svmaccuracyspread[i+1] - svmaccuracyspread[i] < 0.05 and svmaccuracyspread[i+2] - svmaccuracyspread[i] < 0.05 and svmaccuracyspread[i+3] - svmaccuracyspread[i] < 0.05:
            svmInd = i
            break
    for i in range(0,len(knnaccuracyspread)-3):
        if knnaccuracyspread[i+1] - knnaccuracyspread[i] < 0.05 and knnaccuracyspread[i+2] - knnaccuracyspread[i] < 0.05 and knnaccuracyspread[i+3] - knnaccuracyspread[i] < 0.05:
            knnInd = i
            break
    for i in range(0,len(rfaccuracyspread)-3):
        if rfaccuracyspread[i+1] - rfaccuracyspread[i] < 0.05 and rfaccuracyspread[i+2] - rfaccuracyspread[i] < 0.05 and rfaccuracyspread[i+3] - rfaccuracyspread[i] < 0.05:
            rfaInd = i
            break
        
    inds = [logInd, svmInd, knnInd, rfaInd]
    minInd = np.where(np.array(inds) == min(inds))

    optimalFeatNum = inds[minInd[0][0]] + 1

    if minInd[0][0] == 0:
        optimalModel = "log"
    elif minInd[0][0] == 1:
        optimalModel = "svm"
    elif minInd[0][0] == 2:
        optimalModel = "knn"
    elif minInd[0][0] == 3:
        optimalModel = "rfa"


print(optimalFeatNum)
print(optimalModel) #model to be used for snippet


optimalFeatInds = sfsInds[0:optimalFeatNum]
print(optimalFeatInds)

featList = [] #features to be used for snippet
for i in optimalFeatInds:
    featList.append(feature_list[i])
    
print(featList)

5
log
[3, 0, 1, 2, 3]
['SpectralFlatness', 'SpectralCentroid', 'SpectralCrestFactor', 'SpectralDecrease', 'SpectralFlatness']


[4, 4, 4, 3]
(array([3], dtype=int64),)
3
rfa


In [109]:
def snippetGen(path, features, model, chunk, hop):
    
    feature_list = features
    feature_names = features
    
    snipImports = [
        'from scipy.io.wavfile import read',
        'import os',
        'import pandas as pd',
        'import numpy as np',
        'import math',
        'import matplotlib.pyplot as plt',
        'from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg',
        'import pyACA',
        'from sklearn.model_selection import StratifiedKFold, KFold',
        'from sklearn.model_selection import train_test_split',
        'from sklearn.linear_model import LogisticRegression',
        'from sklearn import svm',
        'from sklearn.neighbors import KNeighborsClassifier',
        'from sklearn.ensemble import RandomForestClassifier',
        'from sklearn.preprocessing import StandardScaler',
        'from sklearn.pipeline import make_pipeline',
        'from sklearn.metrics import accuracy_score',
        'from itertools import combinations',
        'from sklearn.base import clone',
        'from sklearn.metrics import accuracy_score',
        'from sklearn.metrics import balanced_accuracy_score',
        'from sklearn.metrics import confusion_matrix'
    ]
    
    snipReadData = [
        'def readData(path, chunk, hop):',
        '    #chunk = values[\'-CHUNK-\']',
        '    #hop = (1 - (values[\'-HOP-\']) / 100) * chunk',
        '    class_names = os.listdir(path)',
        '    filenames_1 = os.listdir(path + \'/\' + class_names[1])',
        '    filenames_2 = os.listdir(path + \'/\' + class_names[2])',
        '    num_files = len(filenames_1) + len(filenames_2)',
        '    time_series_a = []  # initializing variables',
        '    time_series_b = []',
        '    bool_check = True',
        '    for i in range(num_files):  # step 1 progress bar',
        '        if not sg.one_line_progress_meter(\'File Reading Progress\', i + 1, num_files, \'step 1 of 3: file reading\'):',
        '            break',
        '        if i < len(filenames_1):    # processing class A',
        '            [current_fs1, x] = read(path + \'/\' + class_names[1] + \'/\' + filenames_1[i])',
        '            x = x[:, 0]  # grabbing only channel 1 of recording',
        '            time_series_a.extend(x)',
        '            if bool_check:  # grabbing the first returned sampling-rate to check subsequent values against',
        '                sampling_rate_check = current_fs1',
        '                bool_check = False',
        '            else:',
        '                if sampling_rate_check != current_fs1:',
        '                    raise Warning(\'Please make sure all files have the same sampling rate\')',
        '        else:   # processing class B',
        '            [current_fs2, x2] = read(values[\'-PATH-\'] + \'/\' + class_names[2] + \'/\' + filenames_2[i - len(filenames_1)])',
        '            x2 = x2[:, 0]  # grabbing only channel 1',
        '            time_series_b.extend(x2)',
        '            if sampling_rate_check != current_fs2:',
        '                raise Warning(\'Please make sure all files have the same sampling rate\')',
        '    return chunk, hop, time_series_a, time_series_b, sampling_rate_check'
    ]
    
    snipGetFeatures = [
        'def getFeatures(time_series_a, time_series_b, chunk, hop, sampling_rate):',
        '    def helpGetFeatures(feature, file, f_s):',
        '        [vsf, t] = pyACA.computeFeature(feature, file, f_s, iBlockLength=chunk, iHopLength=hop)',
        '        return vsf',
        '    time_series_a = np.array(time_series_a)     # initializing variables',
        '    time_series_b = np.array(time_series_b)',
        '    features_a = []',
        '    features_b = []',
        '    for f in range(len(feature_names)):',
        '        if not sg.one_line_progress_meter(\'Feature Extraction Progress\', f + 1, len(feature_names),',
        '                                          \'step 2 of 3: feature extraction\'):',
        '            break',
        '        current_feature_a = helpGetFeatures(feature_list[f], time_series_a, sampling_rate)',
        '        features_a.append(current_feature_a)',
        '        current_feature_b = helpGetFeatures(feature_list[f], time_series_b, sampling_rate)',
        '        features_b.append(current_feature_b)',
        '    return features_a, features_b'
    ]
    
    snipFormatData = [
        'def formatData():',
        '    data1 = np.ones(len(features_list_a[0]))',
        '    data2 = np.zeros(len(features_list_b[0]))',
        '    data3 = np.concatenate((data1, data2))',
        '    data5 = []',
        '    for d in range(len(feature_names)):',
        '        data4 = np.concatenate([features_list_a[d], features_list_b[d]])',
        '        data5.append(data4)',
        '    categories = pd.DataFrame(data3)',
        '    predictors = pd.DataFrame(data5).transpose()',
        '    predictors.columns = feature_names',
        '    predictors_scaled = predictors.copy()  # normalization of audio features',
        '    for i in feature_names:',
        '        predictors_scaled[i] = (predictors_scaled[i] - predictors_scaled[i].min())'+ str(' / \\'),
        '                               (predictors_scaled[i].max() - predictors_scaled[i].min())',
        '    training_categories, testing_categories, training_predictors, testing_predictors \\',
        '        = train_test_split(categories, predictors_scaled, test_size=.2, random_state=25)',
        '    return training_categories, testing_categories, training_predictors, testing_predictors'
        
    ]
    
    snipInitializeModels = [
        'def initializeModels():',
        '    model1 = LogisticRegression(solver=\'lbfgs\', max_iter=200)',
        '    model2 = svm.SVC()',
        '    model3 = KNeighborsClassifier(n_neighbors=3)',
        '    model4 = RandomForestClassifier(max_depth=2, random_state=0)',
        '    return model1, model2, model3, model4',
        '',
        '[model1, model2, model3, model4] = initializeModels()'
    ]
    
    if model == 'log':
        snipOptimalModel = 'optimalModel = model1'
    elif model == 'svm':
        snipOptimalModel = 'optimalModel = model2'
    elif model == 'knn':
        snipOptimalModel = 'optimalModel = model3'
    elif model == 'rfa':
        snipOptimalModel = 'optimalModel = model4'
    
    snipExecution = [
        '[chunk, hop, time_series_a, time_series_b, sampling_rate_check] = readData(\''+str(path)+'\', '+str(chunk)+', '+str(hop)+')',
        '[features_a, features_b] = getFeatures(time_series_a, time_series_b, chunk, hop, sampling_rate_check)',
        '[training_categories, testing_categories, training_predictors, testing_predictors] = formatData(features_a, features_b)',
        'optimalModel.fit(training_predictors, training_categories)',
        'predictions = optimalModel.predict(testing_predictors)',
        'cm = confusion_matrix(testing_categories, predictions)',
        'pd.DataFrame(cm)',
        'accuracy = (cm[0,0]+cm[1,1]) / sum(sum(cm))'
    ]
    
    print(*snipImports, sep = '\n')
    print('\n')
    print(*snipReadData, sep = '\n')
    print('\n')
    print(*snipGetFeatures, sep = '\n')
    print('\n')
    print(*snipFormatData, sep = '\n')
    print('\n')
    print(*snipInitializeModels, sep = '\n') 
    print('\n')
    print(snipOptimalModel)
    print('\n')
    print(*snipExecution, sep = '\n')
    
    

    
 

In [110]:
snippetGen('/Users/William/Desktop/AudioTech_2/id_00/',featList,optimalModel,5000,2500)

from scipy.io.wavfile import read
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pyACA
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from itertools import combinations
from sklearn.base import clone
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix


def readData(path, chunk, hop):
    #chunk = values['-CHUNK-']
    #hop = (1 - (values['-HOP-']) / 100) * chunk
    class_names = os.listdir(path)
    filenames_1

In [111]:
from scipy.io.wavfile import read
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pyACA
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from itertools import combinations
from sklearn.base import clone
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix


def readData(path, chunk, hop):
    #chunk = values['-CHUNK-']
    #hop = (1 - (values['-HOP-']) / 100) * chunk
    class_names = os.listdir(path)
    filenames_1 = os.listdir(path + '/' + class_names[1])
    filenames_2 = os.listdir(path + '/' + class_names[2])
    num_files = len(filenames_1) + len(filenames_2)
    time_series_a = []  # initializing variables
    time_series_b = []
    bool_check = True
    for i in range(num_files):  # step 1 progress bar
        if not sg.one_line_progress_meter('File Reading Progress', i + 1, num_files, 'step 1 of 3: file reading'):
            break
        if i < len(filenames_1):    # processing class A
            [current_fs1, x] = read(path + '/' + class_names[1] + '/' + filenames_1[i])
            x = x[:, 0]  # grabbing only channel 1 of recording
            time_series_a.extend(x)
            if bool_check:  # grabbing the first returned sampling-rate to check subsequent values against
                sampling_rate_check = current_fs1
                bool_check = False
            else:
                if sampling_rate_check != current_fs1:
                    raise Warning('Please make sure all files have the same sampling rate')
        else:   # processing class B
            [current_fs2, x2] = read(values['-PATH-'] + '/' + class_names[2] + '/' + filenames_2[i - len(filenames_1)])
            x2 = x2[:, 0]  # grabbing only channel 1
            time_series_b.extend(x2)
            if sampling_rate_check != current_fs2:
                raise Warning('Please make sure all files have the same sampling rate')
    return chunk, hop, time_series_a, time_series_b, sampling_rate_check


def getFeatures(time_series_a, time_series_b, chunk, hop, sampling_rate):
    def helpGetFeatures(feature, file, f_s):
        [vsf, t] = pyACA.computeFeature(feature, file, f_s, iBlockLength=chunk, iHopLength=hop)
        return vsf
    time_series_a = np.array(time_series_a)     # initializing variables
    time_series_b = np.array(time_series_b)
    features_a = []
    features_b = []
    for f in range(len(feature_names)):
        if not sg.one_line_progress_meter('Feature Extraction Progress', f + 1, len(feature_names),
                                          'step 2 of 3: feature extraction'):
            break
        current_feature_a = helpGetFeatures(feature_list[f], time_series_a, sampling_rate)
        features_a.append(current_feature_a)
        current_feature_b = helpGetFeatures(feature_list[f], time_series_b, sampling_rate)
        features_b.append(current_feature_b)
    return features_a, features_b


def formatData():
    data1 = np.ones(len(features_list_a[0]))
    data2 = np.zeros(len(features_list_b[0]))
    data3 = np.concatenate((data1, data2))
    data5 = []
    for d in range(len(feature_names)):
        data4 = np.concatenate([features_list_a[d], features_list_b[d]])
        data5.append(data4)
    categories = pd.DataFrame(data3)
    predictors = pd.DataFrame(data5).transpose()
    predictors.columns = feature_names
    predictors_scaled = predictors.copy()  # normalization of audio features
    for i in feature_names:
        predictors_scaled[i] = (predictors_scaled[i] - predictors_scaled[i].min()) / \
                               (predictors_scaled[i].max() - predictors_scaled[i].min())
    training_categories, testing_categories, training_predictors, testing_predictors \
        = train_test_split(categories, predictors_scaled, test_size=.2, random_state=25)
    return training_categories, testing_categories, training_predictors, testing_predictors


def initializeModels():
    model1 = LogisticRegression(solver='lbfgs', max_iter=200)
    model2 = svm.SVC()
    model3 = KNeighborsClassifier(n_neighbors=3)
    model4 = RandomForestClassifier(max_depth=2, random_state=0)
    return model1, model2, model3, model4

[model1, model2, model3, model4] = initializeModels()


optimalModel = model1


[chunk, hop, time_series_a, time_series_b, sampling_rate_check] = readData('/Users/William/Desktop/AudioTech_2/id_00/', 5000, 2500)
[features_a, features_b] = getFeatures(time_series_a, time_series_b, chunk, hop, sampling_rate_check)
[training_categories, testing_categories, training_predictors, testing_predictors] = formatData(features_a, features_b)
optimalModel.fit(training_predictors, training_categories)
predictions = optimalModel.predict(testing_predictors)
cm = confusion_matrix(testing_categories, predictions)
pd.DataFrame(cm)
accuracy = (cm[0,0]+cm[1,1]) / sum(sum(cm))

IndexError: list index out of range